In [ ]:

# Project Requirements & Ideas

# test

# 1. Initial Data Exploration
#   - Where the dataset come from?
#   - What is the data about?
#   - When it was collected, and over what time period
#   - How many rows and columns
#   - What are the types of the columns
#   - Some basic statistics and plots
#     a. Plot / Pie with Major distribution
#     b. Plot with race distribution
#     c. Plot with work_exp
#     d. Plot / Pie with work industry
#     e. more??

# 2. Data Cleaning
#   - How much missing data
#   - what's the form of missing data?
#   - where's the missing data?
#   - what was done about the missing data?

# 3. Data exploration and visualization
#   - various plots that are appropiate for the data.
#   1. ???
#   - For every plot, the question that motivated it and what you learned from it.

# 4. Propose ML approach
#   - Why specific algorithm/s?
#   - Is there something about the data that has driven that decision?
#   - Use at least 2 algorithms per person with solid alalysis of hyperparameters.

# 5. Evaluate Performance
#   - Why specific metric?
#   - Are there alternatives?
#   - Use at least 2 metrics with solid explanation and performance under that metric.

# 6. Present Results.
#   - Visualize the results.
#   - Explain plots and results carefully.
#   - Provide evidence of the results supporting or not supporting the goal.
#   - Touth on all components in the data science process.
#   - Did we achieve our goal?
#   - What we could have done better?
#   - What else could we do?
#   - What would be most impactful next step and why?

# 7. Final Report.
#   - Turn in code and report document (word/pdf).
#   - Prepare the presentation in PowerPoint.
#   - Each people should highlight its contribution.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
%matplotlib inline

df=pd.read_csv('/kaggle/input/mba-admission-dataset/MBA.csv')
print(df.head())

   application_id  gender  international   gpa       major      race   gmat  \
0               1  Female          False  3.30    Business     Asian  620.0   
1               2    Male          False  3.28  Humanities     Black  680.0   
2               3  Female           True  3.30    Business       NaN  710.0   
3               4    Male          False  3.47        STEM     Black  690.0   
4               5    Male          False  3.35        STEM  Hispanic  590.0   

   work_exp          work_industry admission  
0       3.0     Financial Services     Admit  
1       5.0  Investment Management       NaN  
2       5.0             Technology     Admit  
3       6.0             Technology       NaN  
4       5.0             Consulting       NaN  
